In [1]:
import cv2
import numpy as np

def load_gray_tiff(path: str) -> np.ndarray:
    """
    Load 8-bit grayscale TIFF using OpenCV
    """
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Gagal load image: {path}")
    return img.astype(np.float32)


def mask_valid(*imgs, min_valid=2):
    mask = np.ones_like(imgs[0], dtype=bool)
    for im in imgs:
        mask &= (im >= float(min_valid))
    return mask


def estimate_uter(deltaU, phi_high, mask, exclude_top_percent=0.1):
    """
    Estimate (Uter - Ulow) from brightest region
    excluding top 0.1% intensities
    """
    safe = mask & np.isfinite(deltaU)
    vals = phi_high[safe]

    p_hi = 100.0 - exclude_top_percent
    p_lo = p_hi - 0.4

    thr_hi = np.percentile(vals, p_hi)
    thr_lo = np.percentile(vals, p_lo)

    band = safe & (phi_high >= thr_lo) & (phi_high <= thr_hi)

    if np.sum(band) == 0:
        return float(np.nanmax(deltaU[safe]))

    return float(np.nanmedian(deltaU[band]))


def rs_from_ratio_cv2(phi_low_path,
                      phi_high_path,
                      Uth,
                      J_inj,
                      min_valid=2,
                      exclude_top_percent=0.1):

    phi_low  = load_gray_tiff(phi_low_path)
    phi_high = load_gray_tiff(phi_high_path)

    mask = mask_valid(phi_low, phi_high, min_valid=min_valid) \
           & (phi_low > 0) & (phi_high > 0)

    deltaU = np.full_like(phi_low, np.nan, dtype=np.float32)
    deltaU[mask] = Uth * np.log(phi_high[mask] / phi_low[mask])

    Uter_minus_Ulow = estimate_uter(deltaU, phi_high, mask, exclude_top_percent)

    Rs = np.full_like(deltaU, np.nan, dtype=np.float32)
    Rs[mask] = (Uter_minus_Ulow - deltaU[mask]) / float(J_inj)

    # c(r) relatif
    c_rel = np.full_like(phi_low, np.nan, dtype=np.float32)
    c_rel[mask] = phi_low[mask] / np.nanmedian(phi_low[mask])

    return {
        "Rs_map": Rs,
        "deltaU": deltaU,
        "Uter_minus_Ulow": Uter_minus_Ulow,
        "c_rel": c_rel
    }

In [3]:
result = rs_from_ratio_cv2(
    phi_low_path = r"C:\Users\Ghozy Abror\OneDrive - Institut Teknologi Bandung\Karirku\UNSW\Thesis\Coding\EL_Cell\10084_35_1_01272020_20\10084_35_1_01272020_20_cell_000.tiff",
    phi_high_path = r"C:\Users\Ghozy Abror\OneDrive - Institut Teknologi Bandung\Karirku\UNSW\Thesis\Coding\EL_Cell\10084_35_1_01272020_80\10084_35_1_01272020_80_cell_000.tiff",
    Uth=0.0259,        # bisa treat sebagai effective parameter
    J_inj=0.05         # A/cm² (I_inj / area_cell)
)

print("Median Rs:", np.nanmedian(result["Rs_map"]))

Median Rs: 0.15358216


In [1]:
import os
import glob
import cv2
import numpy as np

try:
    from scipy.special import lambertw
except ImportError as e:
    raise ImportError("Butuh scipy untuk lambertw. Install: pip install scipy") from e


def load_gray_tiff_cv2(path: str) -> np.ndarray:
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Gagal load image: {path}")
    return img.astype(np.float32)


def mean_intensity_excluding_padding(img: np.ndarray, min_valid: int = 2) -> float:
    """
    Hitung mean intensity dengan buang pixel 0-1 (padding/bezel).
    """
    m = img >= float(min_valid)
    if np.sum(m) == 0:
        return float("nan")
    return float(np.mean(img[m]))


def compute_CPRA(phi_ref: float, J0: float, nid: float, Vth: float, rref_p: float) -> float:
    """
    Eq (9) paper: CPRA dihitung dari sel referensi pakai Lambert-W (main branch W0).
    CPRA = (J0 rref_p Phi_ref)/(nid Vth) * [ W0( (J0 rref_p)/(nid Vth) * exp(J0 rref_p/(nid Vth)) ) ]^{-1}
    """
    a = (J0 * rref_p) / (nid * Vth)  # dimensionless
    W = lambertw(a * np.exp(a), k=0)  # main branch
    W = np.real(W)
    if W <= 0:
        raise ValueError("LambertW menghasilkan nilai non-positif. Cek parameter J0, nid, rref_p.")
    CPRA = (J0 * rref_p * phi_ref) / (nid * Vth * W)
    return float(CPRA)


def rp_from_mean_intensity(phi_i: float, CPRA: float, Jc: float, J0: float, nid: float, Vth: float) -> float:
    """
    Eq (10) paper:
    r_i,p = (nid Vth ln(phi_i/CPRA)) / (Jc - J0 * (phi_i/CPRA))
    """
    if not np.isfinite(phi_i) or phi_i <= 0:
        return float("nan")

    x = phi_i / CPRA
    if x <= 0:
        return float("nan")

    num = nid * Vth * np.log(x)
    den = Jc - J0 * x

    # Untuk kasus shunt kuat, den bisa mendekati 0 atau negatif -> tidak fisik untuk PRA
    if den <= 0:
        return float("nan")

    rp = num / den
    # rp harus positif (Ω·cm²). Jika negatif -> parameter/assumption tidak cocok.
    if rp <= 0:
        return float("nan")

    return float(rp)


def estimate_rparallel_per_cell(
    folder_low_cells: str,
    Jc: float,                 # A/cm^2
    J0: float,                 # A/cm^2
    nid: float = 1.2,
    Vth: float = 0.0259,       # kT/q at ~300K (tanpa n). Di rumus pakai nid*Vth, jadi Vth di sini adalah (kT/q).
    rref_p: float = 1e6,       # Ω·cm^2 (sel referensi "baik" -> besar)
    min_valid_intensity: int = 2,
    pattern: str = "*_cell_*.tif*"
):
    """
    Menghasilkan array:
    cell_name, mean_intensity, rp (Ω·cm²)

    Asumsi sesuai PRA:
    - Low current: Rs & Rcon diabaikan (Jc < ~10% Jsc) :contentReference[oaicite:2]{index=2}
    - Defect dominan mengubah Rp, bukan Rs
    - J0 & nid sama untuk semua cell
    """
    paths = sorted(glob.glob(os.path.join(folder_low_cells, pattern)))
    if not paths:
        raise FileNotFoundError(f"Tidak ada file cocok pattern {pattern} di {folder_low_cells}")

    # 1) mean intensity per cell
    cell_names = []
    phi_means = []
    for p in paths:
        img = load_gray_tiff_cv2(p)
        phi = mean_intensity_excluding_padding(img, min_valid=min_valid_intensity)
        cell_names.append(os.path.basename(p))
        phi_means.append(phi)

    phi_means = np.array(phi_means, dtype=np.float64)

    # 2) pilih reference cell: mean intensity tertinggi (best cell)
    idx_ref = int(np.nanargmax(phi_means))
    phi_ref = float(phi_means[idx_ref])

    # 3) hitung CPRA dari Eq (9)
    CPRA = compute_CPRA(phi_ref=phi_ref, J0=J0, nid=nid, Vth=Vth, rref_p=rref_p)

    # 4) hitung rp tiap cell (Eq 10)
    rps = []
    for phi in phi_means:
        rp = rp_from_mean_intensity(phi_i=float(phi), CPRA=CPRA, Jc=Jc, J0=J0, nid=nid, Vth=Vth)
        rps.append(rp)

    rps = np.array(rps, dtype=np.float64)

    result = {
        "cell_names": cell_names,
        "phi_mean": phi_means,
        "rp_ohm_cm2": rps,
        "ref_cell": cell_names[idx_ref],
        "phi_ref": phi_ref,
        "CPRA": CPRA
    }
    return result


# -------------------------
# CONTOH PAKAI
# -------------------------
if __name__ == "__main__":
    folder_low = r"C:\Users\Ghozy Abror\OneDrive - Institut Teknologi Bandung\Karirku\UNSW\Thesis\Coding\EL_Cell\10084_35_1_01272020_20"

    # Kamu isi ini dari arus injeksi / area cell:
    # contoh: I = 0.6 A, area cell = 243 cm^2 => Jc = 0.00247 A/cm^2
    Jc = 0.0025

    # Parameter model (kalau belum ada, isi dulu untuk uji pipeline):
    nid = 1.2
    Vth = 0.0259
    J0 = 1e-9
    rref_p = 1e6

    out = estimate_rparallel_per_cell(
        folder_low_cells=folder_low,
        Jc=Jc,
        J0=J0,
        nid=nid,
        Vth=Vth,
        rref_p=rref_p
    )

    print("Reference cell:", out["ref_cell"])
    print("CPRA:", out["CPRA"])
    # Tampilkan 5 pertama
    for n, phi, rp in list(zip(out["cell_names"], out["phi_mean"], out["rp_ohm_cm2"]))[:5]:
        print(n, "phi_mean=", phi, "rp(ohm*cm2)=", rp)

Reference cell: 10084_35_1_01272020_20_cell_020.tiff
CPRA: 112.51392364501953
10084_35_1_01272020_20_cell_000.tiff phi_mean= 80.94764709472656 rp(ohm*cm2)= nan
10084_35_1_01272020_20_cell_001.tiff phi_mean= 86.95196533203125 rp(ohm*cm2)= nan
10084_35_1_01272020_20_cell_002.tiff phi_mean= 89.15899658203125 rp(ohm*cm2)= nan
10084_35_1_01272020_20_cell_003.tiff phi_mean= 85.58997344970703 rp(ohm*cm2)= nan
10084_35_1_01272020_20_cell_004.tiff phi_mean= 85.47925567626953 rp(ohm*cm2)= nan
